In [124]:
import json
import math
import humanize

In [125]:
def load_json(file):
    f = open(file, 'r')
    return json.load(f)

In [126]:
def write_json(data):
    encoded_data = json.JSONEncoder(indent=2).encode(data)
    
    f = open('mud_of_babel_data.json', 'w')
    f.write(encoded_data)

In [169]:
def doors_dict_hex():
    return {
        "S": "South",
        "N": "North",
        "SE": "SouthEast",
        "NW": "NorthWest",
        "SW": "SouthWest",
        "NE": "NorthEast"
    }

def doors_dict_square():
    return {
        "S": "South",
        "N": "North",
        "E": "East",
        "W": "West"
    }

In [170]:
def build_basic_game_info():
    return {
        'gameOver' : False,
        'introText' : '\n\nWelcome to the MUD of Babel!\n',
        'outroText' : 'Thanks For playing!',
        'player' : {
            'currentLocation' : '',
            'inventory' : {},
            'name' : '',
            'lightSource' : False
        },
        'map': {}
    }

In [171]:
def build_room(key, name, description, exits={}, interactables={}, items={}):
    return {
        key: {
            'firstVisit' : True,
            'displayName' : name,
            'description' : description,
#             'setup' : 'function(){end();}',
            'interactables' : interactables,
            'items' : items,
            'exits' : exits
        }
    }


In [173]:
def build_entrance_exit_description(name, destination, text, short=False):
    exit = {
        destination : {
            'displayName' : name,
            'destination' : destination
        }
    }
    entrance = '[%s] %s' % (name.ljust(10,"|").rjust(11,"|") if short == False else name.ljust(2,"|").rjust(3,"|"), text)
    return entrance, exit

In [242]:
def build_shelf_entrance_exit(shelf, room_code, direction):
    shelf_name = "%s shelf" % (shelf['subject'])
    shelf_code = "%s-%s" % (room_code, shelf['id'])
    return build_entrance_exit_description(name=direction, destination=shelf_code, text=shelf_name)

In [245]:
def build_shelves(shelves, room_subject, room_code):
    rooms = {}
    for index, shelf in enumerate(shelves):
        exits = {}
        entrances = []
        shelf_name = "%s shelf" % shelf['subject']
        shelf_code = "%s-%s" % (room_code, shelf['id'])
        shelf_description = "Description for shelf %s in %s Room.\n" % (shelf['subject'], room_subject)
        # exit to main room lobby
        if index == 0:
            entrance, exit = build_entrance_exit_description(name="Main", destination=room_code, text="%s Room entrance" % room_subject)
            entrances.append(entrance)
            exits.update(exit)
        # exit to previous shelf
        if index > 0:
            entrance, exit = build_shelf_entrance_exit(shelves[index-1], room_code, "Previous")
            entrances.append(entrance)
            exits.update(exit)
        # exit to next shelf
        if index < len(shelves)-1:
            entrance, exit = build_shelf_entrance_exit(shelves[index+1], room_code, "Next")
            entrances.append(entrance)
            exits.update(exit)
        shelf_description = '%s\n%s' % (shelf_description, '\n'.join(entrances))
        rooms.update(build_room(key=shelf_code, name=shelf_name, description=shelf_description, exits=exits))
    return rooms

In [236]:
def build_floor_room(room, floor_code):
    entrances = []
    exits = {}
    rooms = {}
    room_code =  "%s-%s" % (floor_code, room['id'])
    room_name = '%s Room entrance' % room['subject']
    shelf_list = list(room['children'].values())
    shelf_count = len(shelf_list)
    shelf_text = 'shelves' if shelf_count != 1 else 'shelf'
    shelf_description = ""
    if shelf_count > 0:
        first_shelf = shelf_list[0]
        first_shelf_code = "%s-%s" % (room_code, first_shelf['id'])
        first_shelf_name = "%s shelf" % (first_shelf['subject'])
        shelf_description = "The first shelf you can see is %s. " % first_shelf['subject']
        entrance, exit = build_entrance_exit_description(name="Shelf", destination=first_shelf_code, text=first_shelf_name)
        entrances.append(entrance)
        exits.update(exit)
        rooms.update(build_shelves(shelves=shelf_list, room_code=room_code, room_subject=room['subject']))
    room_description = "This room has %s %s organized linearly from the entrance towards the back. %sYou can exit to the floor foyer.\n" % (humanize.apnumber(shelf_count), shelf_text, shelf_description)
    entrance, exit = build_entrance_exit_description(name='Exit', destination=floor_code, text='Floor foyer')
    entrances.append(entrance)
    exits.update(exit)
    room_description = '%s\n%s' % (room_description, '\n'.join(entrances))
    rooms.update(build_room(key=room_code, name=room_name, description=room_description, exits=exits))
    return rooms

In [175]:
def build_floor(subject, code, floor, children, shape, is_top=False):
    '''
    The N-th floor of a building with up to eight doorways
    Doorways are organized in a S/N/E/W/SE/NW/SW/NE order
    Each floor is shaped depending on the shape of the building
    '''
    _child_count = len(children)
    doors_list = doors_dict_hex() if shape == 'hexagonal' else doors_dict_square()
    floor_name = "%s Building, %s Floor" % (subject, humanize.ordinal(floor))
    floor_code = ("%s_%s" % (floor, code) if floor > 1 else code)
    
    exits = {}
    rooms = {}

    be_str = 'are' if _child_count != 1 else 'is'
    room_str = 'rooms' if _child_count != 1 else 'room'
    description = 'You are in the %s floor. There %s %s %s:\n' % (humanize.ordinal(floor), be_str, humanize.apnumber(_child_count), room_str)
    entrances = []
    for index, item in enumerate(children):
        key, val = item
        exit_geo =  list(doors_dict().values())[index]
        exit_key =  "%s-%s" % (floor_code, val['id'])
        entrance, exit = build_entrance_exit_description(name=exit_geo, destination=exit_key, text=('%s Room' % val['subject']))
        entrances.append(entrance)
        exits.update(exit)
        
        room = build_floor_room(room=val, floor_code=floor_code)

        rooms.update(room)

    if floor > 1:
        floor_code = '%s_%s' % (floor, code)
        down_code = ("%s_%s" % (floor-1, code)) if floor-1 > 1 else code
        entrance, exit = build_entrance_exit_description(name='Down', destination=down_code, text=('%s floor' % humanize.ordinal(floor-1)))
        exits.update(exit)
        entrances.append(entrance)

    if (is_top is not False):
        # still floors left
        up_code = "%s_%s" % ((floor+1), code)
        entrance, exit = build_entrance_exit_description(name='Up', destination=up_code, text=('%s floor' % humanize.ordinal(floor+1)))
        exits.update(exit)
        entrances.append(entrance)

    description = '%s\n%s' % (description, '\n'.join(entrances))
    rooms.update(build_room(key=floor_code, name=floor_name, description=description, exits=exits))
    return rooms

In [176]:
def build_path_to_building(code, building_name, building_code, shape, stories):
    name = '%s Building Front Porch' % building_name
    entrances = [
        '[%s] Main Library Grounds' % ('Main'.ljust(6,"|").rjust(7,"|")),
        '[%s] Enter Building' % ('Enter'.ljust(6,"|").rjust(7,"|"))
    ]
    exits = {
        'MAIN' : {
            'displayName' : 'Main',
            'destination' : 'MAIN'
        },
        'Building' : {
            'displayName' : 'Enter',
            'destination' : building_code
        }

    }
    adjective = 'small' if stories < 5 else 'tall'
    description = 'This is a %s, %s story tall, %s-shaped, marble and brick building. The entrance has a marble relief above it with the words “%s” engraved.\n' % (adjective, humanize.apnumber(stories), shape, building_name)
    description = '%s\n%s' % (description, '\n'.join(entrances))
    return build_room(key=code, name=name, description=description, exits=exits)

In [184]:
def build_building(plaza_code, subject, code, children, plaza_subject):
    '''
    Each building is shaped depending on the amount of child classes it has:
    - five or more: hexagonal shape with multiple floors (if six or more children)
    - one to four: square
    '''
    max_rooms_per_floor = 6
    _child_count = len(children)
    shape = "square" if _child_count < 5 else "hexagonal"
    building_code = 'building_%s' % code
    doors_list = doors_dict_hex() if shape == 'hexagonal' else doors_dict_square()
    cardinals = ', '.join(doors_list.values())
    floors = math.ceil(_child_count / max_rooms_per_floor)

    rooms = {}
    
    # the path to the building
    rooms.update(build_path_to_building(code=code, building_name=subject, building_code=building_code, shape=shape, stories=floors))

    entrances = []
    exits = {
        code : {
            'displayName' : 'Exit',
            'destination' : code
        }
    }
    
    '''
    for floors 1-n:
    loop thru children in steps of 8
    '''
    for i in range(0, len(children), max_rooms_per_floor):
        # get the set of max_rooms_per_floor (or fewer)
        floor_children = list(children)[i:(i+max_rooms_per_floor)]
        # check if is last floor
        is_top = (i + max_rooms_per_floor < len(children))
        # build the floor
        rooms.update(build_floor(children=floor_children, shape=shape, code=building_code, floor=int(i/max_rooms_per_floor)+1, subject=subject, is_top=is_top))

    # loop for the first floor exit descriptions
    for index, item in enumerate(list(children)[0:max_rooms_per_floor]):
        key, val = item
        exit_geo =  list(doors_list.values())[index]
        exit_key = "%s-%s" % (building_code, val['id'])
        entrance, exit = build_entrance_exit_description(name=exit_geo, destination=exit_key, text=('%s Room' % val['subject']))
        entrances.append(entrance)
        exits.update(exit)

    if _child_count > max_rooms_per_floor:
        up_code= '%s_%s' % (2, building_code)
        entrance, exit = build_entrance_exit_description(name='Up', destination=up_code, text=('%s floor' % humanize.ordinal(2)))
        exits.update(exit)
        entrances.append(entrance)

    entrances.append('[%s] %s Building Front Porch' % ('Exit'.ljust(10,"|").rjust(11,"|"), subject))

    be_str = 'are' if _child_count != 1 else 'is'
    room_str = 'rooms' if _child_count != 1 else 'room'
    floor_str = 'floors' if floors != 1 else 'floor'
    first_str = ' You are in the first floor.' if floors != 1 else ''
    building_name = "%s Building Main Lobby" % subject
    building_floor = ' in %s %s' % (humanize.apnumber(floors), floor_str) if floors > 1 else ''
    description = 'There %s %s %s throughout this building%s.%s The building is %s-shaped with walls facing %s. The %s in this floor %s:\n' % (be_str, humanize.apnumber(_child_count), room_str, building_floor, first_str, shape, cardinals, room_str, be_str)
    description = '%s\n%s' % (description, '\n'.join(entrances))

    # build room for main lobby
    rooms.update(build_room(key=building_code, name=building_name, description=description, exits=exits))
    return rooms

In [178]:
def build_main_plaza(raw_json, plaza_code, plaza_subject):
    # Main plaza is surrounded by buildings representing all the child classes.
    # The amount of buildings determines the decorations in the plaza and the diameter.
    exits = {}
    max_rooms_per_floor = 6
    _child_count = len(raw_json)
    be_str = 'are' if _child_count != 1 else 'is'
    building_str = 'buildings' if _child_count != 1 else 'building'
    their_str = 'their' if _child_count != 1 else 'its'
    description = 'You are in an plaza surrounded by %s %s conforming %s.\nThe %s %s accessible via %s classification code, visible above the main door:\n' % (_child_count, building_str, plaza_subject, building_str, be_str, their_str)
    entrances = []
    for key, val in raw_json.items():
        exit_key = "%s" % (key)
        stories = '%s %s' % (math.ceil(len(val['children']) / max_rooms_per_floor), 'story' if len(val['children']) <= max_rooms_per_floor else 'stories')
        entrance, exit = build_entrance_exit_description(name=exit_key, destination=exit_key, text=('%s Building' % val['subject']), short=True)
        entrances.append(entrance)
        exits.update(exit)

    description = '%s\n%s' % (description, '\n'.join(entrances))
    return build_room(key=plaza_code, name=plaza_subject, description=description, exits=exits)

In [247]:
'''
Main parsing file
'''
def parse_lcc_file(filename):
    game_info = build_basic_game_info()

    raw_json = load_json(filename)
    
    rooms = {}
    
    plaza_code = 'MAIN'
    plaza_subject = 'Main Library Grounds'
    
    rooms.update(build_main_plaza(raw_json, plaza_code, plaza_subject))

    for child_key, section in raw_json.items():
        section_code = '%s' % (child_key)
        rooms.update(build_building(plaza_code=plaza_code, plaza_subject=plaza_subject, subject=section['subject'], code=section_code, children=section['children'].items()))
    
    first_room = 'MAIN'
    game_info['map'] = rooms
    game_info['player']['currentLocation'] = first_room
    write_json(game_info)

parse_lcc_file('lcc_simple_count.json')

In [206]:
import sys; from PIL import Image; import numpy as np

chars = np.asarray(list(" ░▒▓█"))
chars = np.asarray(list(" .;-:!>7?CO$QHNM"))
# chars = np.asarray(list(" .:-=+*#%@"))
# chars = np.asarray(list(" .'`^\",:;Il!i><~+_-?][}{1)(|/tfjrxnuvczXYUJCLQ0OZmwqpdbkhao*#MW&8%B@$"))

# if len(sys.argv) != 4: print( 'Usage: ./asciinator.py image scale factor' ); sys.exit()
f, SC, GCF, WCF = "castle2.jpg", float(0.1), float(1.5), 6/4

size = 640, 640

img = Image.open(f).convert('LA')
img.thumbnail(size, Image.ANTIALIAS)
S = ( round(img.size[0]*SC*WCF), round(img.size[1]*SC) )
img = np.sum( np.asarray( img.resize(S) ), axis=2)
img -= img.min()
img = (1.0 - img/img.max())**GCF*(chars.size-1)

print( "\n".join( ("".join(r) for r in chars[img.astype(int)]) ) )

...........;;;;;..;;;;.;..................................                  ....................
.......................................................   .                 ....................
..  ......................................................               .......................
                  ..................................... ..            ..........................
                     ...............................                 ...........................
                       ...........................                ..............................
                                 .................             . ...............................
                                  .  ..   ...........             ..............................
                                      ................... .. ..              .  ................
                                       .............       >!7! .!7> 7    .       ..............
.....   .. >::::7O>           